## 검색어 생성

In [1]:
# Schema
from typing_extensions import Annotated
from pydantic import Field, BaseModel

class QueryResult(BaseModel):
    query: Annotated[
        list[str],
        Field(
            ..., 
            max_items=5, 
            min_items=3,
            description="가장 적절한 검색어들의 리스트, 길이 최소 3개/최대 5개", 
        )
    ]

In [2]:
# 프롬프트
from langchain_core.prompts import PromptTemplate

query_template = '''
주어진 제목과 설명을 바탕으로, 가장 연관성이 높은 논문과 데이터셋을 검색하려 합니다.

가장 의미적으로 관련성이 높은 논문과 데이터셋을 검색할 수 있을 쿼리를 만들어 주세요.

[조건]
1. 쿼리는 2~3단어로 구성되어야 합니다.
2. 검색 방식은 쿼리와 정확히 일치하는 내용이 있는 논문이나 데이터를 반환하는 형식입니다.

[Input]
연구 주제: {subject}
연구 설명: {description}

[Output]
가장 적절한 3~5개의 쿼리를 JSON으로 출력해주세요.
'''

query_prompt = PromptTemplate.from_template(query_template)

In [ ]:
from langchain_openai import ChatOpenAI

subject = 'Architectural Urbanism: Melbourne/Seoul - KTA projects'
description = "BACKGROUND: 'Architectural Urbanism: Melbourne/Seoul' was a two-city exhibition funded by RMIT University and the Korean National University, supported by the Australian Government through the Australian International Cultural Council. Kerstin Thompson Architecture (KTA) showed five works - Carrum Downs Police Station, MUMA Gallery, Lake Conneware House, Napier Street Housing and Royal Botanic Gardens Visitor Centre - as one of ten architectural firms (5 from Melbourne, 5 from Seoul) selected to exhibit. The exhibited work included large scale photographs, working drawings and exegetical text. CONTRIBUTION: The exhibition explored architectural approaches that worked 'within the city rather than upon it', architecture that 'intervenes and inserts, rather than overlays or eradicates'. Within this context, the public projects exhibited by KTA explore relationships between interior and exterior, building and street, and the existing and the new. This draws on Thompson's ongoing practice and design research focussing on architecture as a civic endeavour, forging connections between buildings, their surroundings and the people who inhabit them. SIGNIFICANCE: 'Architectural Urbanism' was an international, cross institutional exhibition that received external funding, evidencing significance for the selected works by the 10 participating practices. In Seoul, the exhibition was opened by the deputy Australian Ambassador. The Melbourne exhibition was reviewed in Architecture Australia (AA). These KTA projects have also received multiple awards and media attention including four AIA (Australian Institute of Architects) Victorian Architecture Awards, an IDEA (Interior Design Excellence Awards) award, three book features, critical reviews in AA, and articles in SMH, The Australian, Broadsheet and Archdaily."

prompt = query_prompt.invoke(
    {
        'subject': subject, 
        'description': description,
    }
)

# sLLM
sllm = ChatOpenAI(model='gpt-4o-mini', temperature=0)

structured_sllm = sllm.with_structured_output(QueryResult)
res = structured_sllm.invoke(prompt)


print(res.query)

['Architectural Urbanism', 'Melbourne Seoul', 'KTA projects', 'public architecture', 'civic architecture']
